In [1]:
import pandas as pd
import numpy as np
import re
from sklearn import preprocessing
from keras.layers import Dense
from keras.layers import Embedding
from keras import Sequential
import keras
import pickle
from IPython.display import HTML

Using TensorFlow backend.


In [2]:
def clean_text_lyric(lyric, max_length):
    lyric_sample = lyric.split('</span>')
    lyric_sample = lyric_sample[:-1]
    
    clean_song_lyric = ""
    for sentence in lyric_sample:
        if 'http' not in sentence:
            sentence = re.sub(r'<div .*>','',sentence)
            sentence = re.sub(r'<span .*>','',sentence)
            sentence = sentence.strip()
            if len(sentence) > 0:
                sentence = re.sub(r'[^a-zA-Z0-9<\/>\s]',' ', sentence)
                
                sentence = " <div> " + sentence + " </div> "
                sentence = re.sub(r'<div>\s+<br>','<br> <div> ', sentence)
                sentence = re.sub(r'(\d|\r|\t|\n)','',sentence)

                clean_song_lyric += sentence

    clean_song_lyric = ' <start> ' * (max_length-1) + clean_song_lyric
    return clean_song_lyric

In [3]:
max_length = 10

scrape_lyric = pd.read_csv(r'C:\kuliah\NLP\[TUGASMANDIRINLP][054][131]\scraper.csv')
scrape_lyric = scrape_lyric[['song','title']]
scrape_lyric = scrape_lyric.dropna()
scrape_lyric = scrape_lyric[scrape_lyric.song.str.contains('span')]
scrape_lyric.song = scrape_lyric.song.apply(lambda x: clean_text_lyric(x,max_length))
scrape_lyric = scrape_lyric[~(scrape_lyric.song.str.contains('Belum Ada Lirik'))]
scrape_lyric = scrape_lyric.reset_index(drop=True)

In [4]:
def generate_train_dataset(song_series, window):
    train_data = []
    train_label = []
    for song_string in song_series:
        song_arr = song_string.split()
        len_song_arr = len(song_arr)

        for i in range(0,len_song_arr-window+1):
            train_data.append(' '.join(song_arr[i:i+window-1]))
            train_label.append(song_arr[i+window-1])
    return np.array(train_data), np.array(train_label)

In [5]:
X, y = generate_train_dataset(scrape_lyric.song, window=max_length)
pre = keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-.:;=?@[\\]^_`{|}~\t\n')
pre.fit_on_texts(X.flatten())

vocab_size = len(pre.word_index)
X = pre.texts_to_sequences(X)
y = pre.texts_to_sequences(y)

In [6]:
model = Sequential()
e = Embedding(vocab_size,100 ,input_length=max_length-1, trainable=True)
model.add(e)
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Bidirectional(keras.layers.LSTM(100)))
model.add(keras.layers.BatchNormalization())

model.add(Dense(100, activation='relu'))
model.add(keras.layers.BatchNormalization())

model.add(Dense(vocab_size+1, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [7]:
def batch_data(start,end):
    global X,y,y_input, X_input
    X_input = np.array(X)[start:end]
    y_input = np.array(y)[start:end]
    y_input = keras.utils.to_categorical(y_input,vocab_size+1)


In [8]:
i = 0
max_epoch = 1 # Change this max_epoch
instance_max = len(X)
batch_size = 10000

for i in range(max_epoch):
    start_idx_counter = 0
    print("epoch %d" % (i))
    while start_idx_counter < instance_max:
        batch_data(start_idx_counter, min(start_idx_counter+batch_size,instance_max))
        model.fit(X_input, y_input, epochs=1, batch_size=64) 
        start_idx_counter += batch_size



epoch 0


C:\Users\user\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
10000/10000 [==============================] - 21s 2ms/step - loss: 7.1162
Epoch 1/1
10000/10000 [==============================] - 20s 2ms/step - loss: 5.1499
Epoch 1/1
10000/10000 [==============================] - 20s 2ms/step - loss: 5.0523
Epoch 1/1
10000/10000 [==============================] - 19s 2ms/step - loss: 4.8824
Epoch 1/1
10000/10000 [==============================] - 22s 2ms/step - loss: 4.8264
Epoch 1/1
10000/10000 [==============================] - 21s 2ms/step - loss: 4.6917
Epoch 1/1
10000/10000 [==============================] - 20s 2ms/step - loss: 4.7236
Epoch 1/1
10000/10000 [==============================] - 20s 2ms/step - loss: 4.4530
Epoch 1/1
10000/10000 [==============================] - 19s 2ms/step - loss: 4.5781
Epoch 1/1
10000/10000 [==============================] - 20s 2ms/step - loss: 4.1749
Epoch 1/1
10000/10000 [==============================] - 20s 2ms/step - loss: 4.2947
Epoch 1/1
10000/10000 [==============================] - 20s 2ms/

InvalidArgumentError:  indices[61,8] = 20616 is not in [0, 20616)
	 [[node embedding_1/embedding_lookup (defined at C:\Users\user\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_keras_scratch_graph_5566]

Function call stack:
keras_scratch_graph


In [9]:
model.save('lyric_gen_model.h5')
pickle.dump( pre, open( "tokenizer_pre_lyric.p", "wb" ))

In [10]:
model = keras.models.load_model('lyric_gen_model.h5')
pre = pickle.load( open( "tokenizer_pre_lyric.p", "rb" ) )

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [11]:
def preprocess_predict(X):
    X = pre.texts_to_sequences(X)
    return X

def is_word_in_dict(words, word_idx):
    for word in words.split():
        if word not in word_idx:
            print(word_idx)
            print(word)
            return False
    return True

def generate_song_by_model(model, idx_word, X_sent, total_length, word_idx, total_classes, word_generated):
    string_returned = X_sent
    if is_word_in_dict(X_sent, word_idx):
        X = np.array(preprocess_predict([X_sent]))
        counter = 1

        for i in range(word_generated):
            y_pred = model.predict(X)
            rnd_choice = np.random.choice(range(0,total_classes),size=1,replace=False, p=y_pred[0])
            word_predict = idx_word[rnd_choice[0]]
            string_returned = string_returned + " " + word_predict
            X = ' '.join(string_returned.split()[counter:])
            X = np.array(preprocess_predict([X]))
            counter += 1

        return string_returned

In [13]:
max_length = 10
vocab_size = len(pre.word_index)
seed = "<start> " * (max_length-1)
generated_lyric = generate_song_by_model(model, pre.index_word, seed, max_length, pre.word_index, vocab_size+1,word_generated=50)
HTML(generated_lyric)